In [59]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.externals import joblib
import time
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split


import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)

In [2]:
pruebas = pd.read_csv('properati_dataset_modificado.csv')

In [28]:
sup_min = 9
sup_max = 150

lat_min = -35
lat_max = -34

lon_min = -59
lon_max = -58

precio_min = 5000
precio_max = 1000000

propiedades = pruebas
p = propiedades.loc[  (propiedades.superficie >= sup_max) | (propiedades.superficie <= sup_min) |\
                      (propiedades.lat >= lat_max) | (propiedades.lat <= lat_min) |\
                       (propiedades.lon >= lon_max) | (propiedades.lon <= lon_min),:]

columnas = ['superficie','lat', 'lon']

In [29]:
p.loc[:,columnas].describe()

,superficie,lat,lon
count,4.170000e+03,4170.000000,4170.000000
mean,1.144530e+03,-34.667977,-58.460659
std,5.027828e+04,0.347734,0.320623
min,1.000000e+00,-38.416097,-63.616672
25%,1.670000e+02,-34.756526,-58.632980
50%,2.245681e+02,-34.646625,-58.459830
75%,3.200000e+02,-34.598624,-58.381905
max,3.243242e+06,-29.424917,-57.558500


In [30]:
p.loc[:,columnas].count()

superficie    4170
lat           4170
lon           4170
dtype: int64

## Hay 4000 propiedades con datos "malos" en las pruebas a analizar

## Quizas por eso tenemos muy malos resultados

In [31]:
sup = propiedades.loc[  (propiedades.superficie >= sup_max) | (propiedades.superficie <= sup_min),:]
lat = propiedades.loc[  (propiedades.lat >= lat_max) | (propiedades.lat <= lat_min) ,:]
lon = propiedades.loc[  (propiedades.lon >= lon_max) | (propiedades.lon <= lon_min),:]

columnas = ['superficie','lat', 'lon']


In [32]:
s = sup.loc[:,'id'].count()
l = lat.loc[:,'id'].count()
lo = lon.loc[:,'id'].count()

In [33]:
print("Mal superficie   {}\nMal Latitud   {}\nMal longitud   {}".format(s,l,lo))

Mal superficie   3677
Mal Latitud   70
Mal longitud   637


### El principal problema es la superficie

In [37]:
sup.loc[:,['property_type','superficie','surface_total_in_m2','surface_covered_in_m2']].sample(10)

,property_type,superficie,surface_total_in_m2,surface_covered_in_m2
13157,house,260.0,260.0,130.0
11590,apartment,289.0,289.0,270.0
2081,house,419.0,419.0,70.0
10286,house,340.0,340.0,195.0
4727,house,583.0,583.0,583.0
10951,PH,176.0,176.0,176.0
8590,apartment,295.0,295.0,280.0
825,house,160.0,NaN,160.0
963,house,300.0,NaN,300.0
7348,house,170.0,NaN,170.0


In [52]:
def normalizar_sup(s):
    if s < sup_min:
        return sup_min
    while (s > sup_max):
        s = s / 2
    return s

def normalizar_lat(s):
    if s > lat_min and s < lat_max:
        return s
    return propiedades.loc[:,'lat'].mean()

def normalizar_lon(s):
    if s > lon_min and s < lon_max:
        return s
    return propiedades.loc[:,'lon'].mean()

In [41]:
sup.loc[:,'superficie'] = sup.loc[:,'superficie'].apply(lambda x: normalizar_sup(x))

/home/mati/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [42]:
sup.loc[:,'superficie'].describe()

count    3677.000000
mean      105.439971
std        27.163999
min         9.000000
25%        87.500000
50%       102.500000
75%       125.000000
max       150.000000
Name: superficie, dtype: float64

In [51]:
pruebas = pd.read_csv('properati_dataset_modificado.csv')

In [45]:
pruebas.loc[:,'superficie'] = pruebas.loc[:,'superficie'].apply(lambda x: normalizar_sup(x))
pruebas.loc[:,'superficie'].describe()

count    14166.000000
mean        76.443202
std         34.037244
min          9.000000
25%         47.000000
50%         74.000000
75%        100.000000
max        150.000000
Name: superficie, dtype: float64

In [54]:
pruebas.loc[:,'lat'] = pruebas.loc[:,'lat'].apply(lambda x: normalizar_lat(x))
pruebas.loc[:,'lon'] = pruebas.loc[:,'lon'].apply(lambda x: normalizar_lon(x))
pruebas.loc[:,['lat','lon','superficie']].describe()

,lat,lon,superficie
count,14166.000000,14166.000000,14166.000000
mean,-34.633584,-58.480782,76.443202
std,0.090769,0.124550,34.037244
min,-34.991982,-58.975341,9.000000
25%,-34.651573,-58.510157,47.000000
50%,-34.612545,-58.452145,74.000000
75%,-34.587729,-58.412486,100.000000
max,-34.112857,-58.008589,150.000000


In [55]:
pruebas.to_csv('properati_dataset_modificado2.csv')

In [56]:
propiedades = pd.read_csv('../set_datos_propiedades.csv')

sup_min = 10
sup_max = 150
lat_min = -35
lat_max = -34
lon_min = -59
lon_max = -58
precio_min = 5000
precio_max = 1000000

propiedades = propiedades.loc[(propiedades.price_aprox_usd.notnull()) & (propiedades.superficie.notnull())\
                              & (propiedades.lat.notnull()) & (propiedades.lon.notnull()) & (propiedades.Year >=2016)\
                        & (propiedades.price_aprox_usd <= precio_max) & (propiedades.price_aprox_usd >= precio_min) &\
                         (propiedades.superficie <= sup_max) & (propiedades.superficie >= sup_min) &\
                      (propiedades.lat <= lat_max) & (propiedades.lat >= lat_min) &\
                       (propiedades.lon <= lon_max) & (propiedades.lon >= lon_min),:]

columnas = ['superficie','lat', 'lon','property_type_encoded','seguridad','gimnasio', 'aire', 'pileta', 'cochera']
columnas_precio = columnas + ['price_aprox_usd']

set_pruebas = analizar = pd.read_csv("properati_dataset_modificado2.csv")
set_pruebas.loc[:,'price_usd'] = 0.0

In [57]:
cant = 10

for i in range(cant):
    datos = propiedades.sample(frac = 0.7)
    set_entrenamiento_datos = datos.loc[:,columnas]
    set_entrenamiento_resultado = datos.loc[:,'price_aprox_usd']
    
    tree = DecisionTreeRegressor(criterion = 'friedman_mse', max_depth = 75, max_features = 0.8)
    
    tree.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
    set_pruebas.loc[:,'res'] = set_pruebas.loc[:,columnas].apply(lambda x: tree.predict(x)[0],axis = 1)
    set_pruebas.loc[:,'price_usd'] = set_pruebas.loc[:,'price_usd'] + set_pruebas.loc[:,'res']

In [58]:
set_pruebas.loc[:,'price_usd'] = set_pruebas.loc[:,'price_usd'] / cant

resultado = set_pruebas.loc[:,['id','price_usd']]

resultado.to_csv('prueba_decisionTree_datos_modificados.csv', index = False)

In [60]:
resultado.describe()

,id,price_usd
count,1.416600e+04,14166.000000
mean,2.285382e+06,173559.264280
std,4.290968e+04,105794.360735
min,3.632000e+03,5005.030000
25%,2.271809e+06,105000.000000
50%,2.290284e+06,144000.000000
75%,2.309395e+06,210275.714286
max,2.332338e+06,912800.000000
